# Assignment Exercise 1 (Marks Available for Exercise 1: 7 marks)

This exercise is heavily based on the Computing Lab from week 1. Consider the following 

1. Generate noisy, simulated data for a model where an input $T$ gives output $Z$ via the relationship, $Z = c \sin(T + d)$. (*For your write-up, use c=0.5 and d=-0.25.*)

2. Next, using your simulated data, write a stochastic gradient descent procedure to learn the parameters $(c, d)$ from the simulated values of $(T, Z)$.

3. Produce plots showing loss and parameter values after each optimisation step.

**Mark Scheme for Exercise 1**

Quality of code and commenting. **Total marks available: 4**

  0. No attempt made to create a working algorithm (Mark: 0)
  1. Attempt made to create a working algorithm but the code contains severe flaws and lacks any commenting. 
  2. A credible attempt to produce working code but with some errors and adequate code commenting. 
  3. A correct solution to the problem produced with no errors and adequate code commenting. 
  4. A correct solution to the problem produced with no errors and comprehensive and detailed code commenting.

Appropriate use of graphics and visualisation. **Total marks available: 3**

# Assignment Exercise 2 (Marks Available for Exercise 2: 13 marks)

**USE scikit-learn for the questions on this Exercise.**

The dataset you will use in this exercise comes from a popular machine learning repository that hosts open source datasets for educational and research purposes, the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php). We are going to use random forests for predicting air quality. The description of the dataset can be found [here](https://archive.ics.uci.edu/ml/datasets/Air+Quality).

In [ ]:
import urllib.request
doq = "https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip"
pat_sav = "./AirQualityUCI.zip"
urllib.request.urlretrieve(doq, pat_sav)

In [ ]:
import zipfile
zip = zipfile.ZipFile('./AirQualityUCI.zip', 'r')
for name in zip.namelist():
    zip.extract(name, '.')

In [ ]:
# The .csv version of the file has some typing issues, so we use the excel version
import pandas as pd 
air_quality_full = pd.read_excel('./AirQualityUCI.xlsx', usecols=range(2,15))

We can see some of the rows in the dataset 

In [ ]:
air_quality_full.sample(5)

The target variable corresponds to the CO(GT) variable of the first column. The following columns correspond to the variables in the feature vectors, *e.g.*, PT08.S1(CO) is $x_1$ up until AH which is $x_D$. The original dataset also has a date and a time columns that we are not going to use in this assignment.

### Removing instances 

The dataset has missing values tagged with a -200 value. To simplify the design of the machine learning models in this assignment, we perform the following two operations to the dataset right from the beginning:

* we will remove the rows for which the target variable has missing values. We are doing supervised learning so we need all our data observations to have known target values.

* we will remove features with more than 20% of missing values. 

The code below performs both operations

In [ ]:
# We first remove the rows for which there are missing values in the target feature
air_quality = air_quality_full.loc[air_quality_full.iloc[:, 0]!=-200, :]

In [ ]:
# We now remove the columns (features) for which there are more that 20% of missing values
import numpy as np
ndata, ncols = np.shape(air_quality) # number of data observations and number of columns in the dataframe
pmissing = np.empty(ncols)         # An empty vector that will keep the percentage of missing values per feature
for i in range(ncols):
    pmissing[i] = (air_quality.iloc[:, i]==-200).sum()/ndata # Computes the percentage of missing values per column
air_quality = air_quality.loc[:, pmissing < 0.2]

### Splitting the dataset 

Before designing any machine learning model, we need to set aside the test data. We will use the remaining training data for fitting the model. *It is important to remember that the test data has to be set aside before preprocessing*. 

Any preprocessing that you do has to be done only on the training data and several key statistics need to be saved for the test stage.  Separating the dataset into training and test before any preprocessing has happened help us to recreate the real world scenario where we will deploy our system and for which the data will come without any preprocessing.

Furthermore, we are going to use *hold-out validation* for validating our predictive model so we need to further separate the training data into a training set and a validation set.

We split the dataset into a training set, a validation set and a test set. The training set will have 70% of the total observations, the validation set 15% and the test set, the remaining 15%. For making the random selections of the training and validation sets **make sure that you use a random seed that corresponds to the last five digits of your student UCard**. In the code below, I have used 55555 as an example of my random seed.

In [ ]:
np.random.seed(55555)                 # Make sure you use the last five digits of your student UCard as your seed
index = np.random.permutation(ndata)  # We permute the indexes 
N = np.int64(np.round(0.70*ndata))    # We compute N, the number of training instances
Nval = np.int64(np.round(0.15*ndata)) # We compute Nval, the number of validation instances   
Ntest = ndata - N - Nval              # We compute Ntest, the number of test instances
data_training_unproc = air_quality.iloc[index[0:N], :].copy() # Select the training data
data_val_unproc = air_quality.iloc[index[N:N+Nval], :].copy() # Select the validation data
data_test_unproc = air_quality.iloc[index[N+Nval:ndata], :].copy() # Select the test data

## Preprocessing the data

As mentioned before, the dataset has missing values tagged with a -200 value. Before doing any work with the training data, we want to make sure that we deal properly with the missing values. Furthermore, once we have dealt with the missing values, we want to standardise the training data. 

### Question 2.a: Pipeline for missing values and standardisation (3 marks)

* Employ the `SimpleImputer` method in Scikit-learn to impute the missing values in each column using the mean value of the non-missing values, instead (**1 mark**).

* Standardise the data by substracting the mean value for each feature and dividing the result by the standard deviation of each feature. Employ the `StandardScaler` method (**1 mark**).

* Create a `Pipeline` that you can use to preprocess the data by filling missing values and then standardising the features (**1 mark**).

#### Question 2.a Answer

In [ ]:
# Write your code here

### Question 2.b: Use the Pipeline to fit the training data and transform the validation data (2 marks)

In the previous question, you created a `Pipeline` for applying a `SimpleImputer` and a `StandardScaler`. Use the Pipeline to fit the training data (**1 mark**) and transform the validation data (**1 mark**).

#### Question 2.b Answer

In [ ]:
# Write your code here

## Random forest to predict air quality 

We now use random forests to predict air quality. Remember that the tree ensemble in random forests is built by training individual regression trees on different subsets of the training data and using a subset of the available features. For regression, the prediction is the average of the individual predictions of each tree. Some of the parameters required in the Random Forest implementation in Scikit-learn include:

Some of the additional parameters required in the Random Forest implementation in Scikit-learn include

> **n_estimators** the total number of trees to train<p>
**max_features** number of features to use as candidates for splitting at each tree node. <p>
    **boostrap**: Whether bootstrap samples are used when building trees. If False, the whole dataset is used to build each tree.<p>
   **max_samples**: If bootstrap is True, the number of samples to draw from X to train each base estimator.

### Question 2.c: train a random forest (4 marks)

In this question, you will train a random forest for predicting over the validation data. Use cross-validation over the validation data to select the best set of paramaters for the random forest regressor. Parameters to include in your exploration are **n_estimators**, **max_features** and **max_samples**. Use `np.linspace` or `np.logspace` to define ranges of values to explore for each parameter and create a grid to be assessed over the validation data. Make sure you use the same validation data that was given to you.

* Use `PredefinedSplit` to tell the cross validator which instances to use for training and which ones for validation (**1 mark**).

* Create a grid of values to explore that include a range of at least five values for each parameter **n_estimators**, **max_features** and **max_samples** (**1 mark**). 

* Train a random forest for regression model that uses the grid of parameters you created before. Use `GridSearchCV` to find the best set of parameters by performing cross-validation over the predefined split. (**1 mark**).

* Print the best values in your grid for **n_estimators**, **max_features** and **max_samples** (**1 mark**).

#### Question 2.c Answer

In [ ]:
# Write your code here

### Question 2.d: train a new model over the whole training data and report the prediction over the test set (4 marks)


Now that we have identified the best paramaters of the regression model, we use these parameters to train a new model over the whole training data (`data_training` plus `data_val`). We apply this model to the test set and report the performance.

* Create a new preprocessing pipeline for taking care of the missing values and standardisation over the whole training data (**1 mark**).

* Apply the created preprocessing pipeline to the test data (**1 mark**).

* Fit a random forest regression model to the training data using the best parameters found in Question 2.c (**1 mark**).

* Compute the RMSE over the test data and report the result (**1 mark**).

#### Question 2.d Answer

In [ ]:
# Write your code here